## 1.数据预处理

Data Preprocessing

In [46]:
# 控制调试输出的开关
DEBUG = not False

### 1.1 配置参数 
> 需根据实际文件路径修改
- 需要解析 PDF 文件路径
- 输出文件 CSV 路径
- 输出文件 CSV 表头

In [47]:
CONST_PDF_PATH = "./assets/2024年四川省重点项目名单.pdf"  
CONST_OUTPUT_DIR = "./outputs"
CONST_ORIGIN_HEADERS = ["总序号", "分序号", "项目名称", "建设地址"]
CONST_TARGET_HEADERS = ["项目名称", "建设地址", "一级项目领域", "二级项目领域", "建设性质"]


### 1.2 使用 pdfplumber 解析 PDF 页面数据

In [48]:
# 调试 pdfplumber
import pdfplumber

if False:
    with pdfplumber.open(CONST_PDF_PATH) as pdf:
        page = pdf.pages[0]
        img = page.crop(bbox=[0, 0, 584, 142]).to_image() 
        img.show()  # 显示图像，手动观察并估算坐标

In [49]:
import pdfplumber
import re

# 申明一个(上下文)变量，一级项目领域，类型为字符串
Level_1_Project_Domain = ""
Level_2_Project_Domain = ""
Level_3_Project_Domain = ""

def extract_table_from_pdf(pdf_path):
    result_key_project_list = []  # 存储所有页面的表格数据

    with pdfplumber.open(pdf_path) as pdf:
        # 获取实际总页数
        total_pages = len(pdf.pages)  
        
        # 遍历PDF的每一页
        for page_num in range(0, total_pages):
        # for page_num in range(0, 10):
            page = pdf.pages[page_num]
            print(f"\r正在解析第 {page_num + 1} 页...", end='', flush=True)
            
            # 提取当前页所有表格
            table = page.extract_table(
                # 参考：
                # https://github.com/jsvine/pdfplumber?tab=readme-ov-file#table-extraction-settings
                table_settings={
                    "vertical_strategy": "lines",  # 按竖线识别列边界
                    "horizontal_strategy": "lines",  # 按横线识别行边界
                    "text_y_tolerance": 10,  # 文本垂直方向容错
                    "text_x_tolerance": 5,   # 文本水平方向容错
                }
            )

            # 提取页面标题，解析(项目类型)
            title_text = page.crop(bbox=[0, 0, 584, 130]).extract_text() or ""

            construction_nature = ""

            if title_text.find("新建") != -1 or title_text.find("新开工") != -1 :
                construction_nature = "新建"

            if title_text.find("续建") != -1:
                construction_nature = "续建"
            
            # 使用新函数处理表格数据
            table_data = process_table_rows(table, construction_nature)

            if False:
                print(f"🌟第 {page_num + 1} 页处理后的数据：")
                for row in table_data:
                    print(row)

            # 将处理后的数据添加到总数据列表中
            result_key_project_list.extend(table_data)

        print(f"\r解析完成！总共 {total_pages} 页。")
    
    return result_key_project_list

def process_table_rows(table, construction_nature):
    # 存储当前表格处理后的数据
    # example:
    # [
    #     ["项目名称", "建设地址", "一级项目领域", "二级项目领域", "建设性质(新建/续建)"]
    #     [……], 
    # ]
    result_list = []  

    for index, row in enumerate(table):

        # 跳过表头行
        if index < 2:
            continue  

        # 解构为单元格，总序号、分序号、项目名称、建设地址
        # 移除两侧潜在的空格
        total_seq_cell, sub_seq_cell, project_name_cell, construction_address_cell = [cell.strip() if cell else "" for cell in row]

        # 清理项目名称单元格，去除换行符和括号及其后内容
        # "教育、文化、体育及社会服务（16项）" -> "教育、文化、体育及社会服务"
        project_name_cell = remove_after_first_paren(project_name_cell.replace("\n", ""))

        global Level_1_Project_Domain 
        global Level_2_Project_Domain 
        global Level_3_Project_Domain 

        # total_seq_cell 为 '一二三四五' 其中一个汉字，则使用 project_name_cell 作为 一级项目领域
        if re.match(r'^[一二三四五]$', total_seq_cell):
            Level_1_Project_Domain = project_name_cell
            Level_2_Project_Domain = ""
            Level_3_Project_Domain = ""
            continue

        if re.match(r'^（[一二三四五]）$', total_seq_cell):
            Level_2_Project_Domain = project_name_cell
            Level_3_Project_Domain = ""
            continue

        if total_seq_cell in ['I', 'II', 'III', 'IV', 'V', 'VI']:
            Level_3_Project_Domain = project_name_cell
            continue

        # 跳过建设地址为空的行
        if not construction_address_cell or construction_address_cell == '':
            continue

        # 建设地址的换行符处理为空格
        address_city = construction_address_cell.replace("\n", " ")

        refined_data = [
            project_name_cell, 
            address_city, 
            Level_1_Project_Domain, 
            Level_2_Project_Domain,
            construction_nature
        ]

        if False:
            print(f"🌈已经处理第 {index + 1} 行：{refined_data}")

        result_list.append(refined_data)

    return result_list

def remove_after_first_paren(text):
    # 使用正则表达式匹配第一个左括号及其后面的所有内容并替换为空字符串
    return re.sub(r'（.*', '', text)
    
if False:
    result_key_project_list = extract_table_from_pdf(CONST_PDF_PATH) 

    print(f"🌟所有表格处理后的数据：")
    for row in result_key_project_list:
        print(row)
    

### 1.3 使用 pandas 数据清洗

转换为 DataFrame 并清洗

In [ ]:
import pandas as pd

def convert_to_dataframe(data, columns):
    df = pd.DataFrame(data, columns=columns)

    if False:
        print(df.shape)       # 输出 (2, 2) → 表示 2 行 2 列
        print(df.columns)     # 输出 Index(['姓名', '年龄'], dtype='object') → 列名
        print(df.index)       # 输出 RangeIndex(start=0, stop=2, step=1) → 行索引
        print(df.dtypes)      # 输出各列数据类型（如 姓名: object，年龄: int64）
        print(df.values)      # 输出 numpy 数组形式的原始数据 → [['张三' 20] ['李四' 22]]
    return df

if False:
    result = extract_table_from_pdf(CONST_PDF_PATH) 
    df = convert_to_dataframe(result, CONST_TARGET_HEADERS)

### 1.4 保存结果（CSV）

In [51]:
import os

def save_results(df, pdf_path, output_dir):

    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)

    # 生成输出文件路径
    output_csv = get_output_csv_path(pdf_path, output_dir)

    # 保存为 CSV（兼容更多工具，如 Excel、Python读取）
    # utf-8-sig 解决中文乱码
    df.to_csv(output_csv, index=False, encoding="utf-8-sig") 
    return output_csv

def get_output_csv_path(pdf_path, output_dir):
    """
    根据PDF文件名生成输出CSV文件路径
    
    Args:
        pdf_path: PDF文件路径
        output_dir: 输出目录
    
    Returns:
        生成的CSV文件完整路径
    """

    # 获取PDF文件名（不含扩展名）
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    # 构造输出CSV文件路径
    output_csv = os.path.join(output_dir, f"{pdf_filename}_解析结果.csv")
    return output_csv

if not False:
    result = extract_table_from_pdf(CONST_PDF_PATH) 
    df = convert_to_dataframe(result, CONST_TARGET_HEADERS)
    save_results(df, CONST_PDF_PATH, CONST_OUTPUT_DIR)

解析完成！总共 76 页。
(727, 5)
Index(['项目名称', '建设地址', '一级项目领域', '二级项目领域', '建设性质'], dtype='object')
RangeIndex(start=0, stop=727, step=1)
项目名称      object
建设地址      object
一级项目领域    object
二级项目领域    object
建设性质      object
dtype: object
[['国家实验室' '四川 天府新区' '基础设施项目' '新型基础设施' '续建']
 ['极深地下极低辐射本底前沿物理实验设施' '凉山州 冕宁县' '基础设施项目' '新型基础设施' '续建']
 ['光场调控装置及地方配套项目' '四川 天府新区' '基础设施项目' '新型基础设施' '续建']
 ...
 ['西昌市两河口再生水处理厂工程' '凉山州 西昌市' '生态建设及环境保护项目' '生态修复、环境整治' '新建']
 ['越西河流域河道综合治理工程' '凉山州 越西县' '生态建设及环境保护项目' '生态修复、环境整治' '新建']
 ['汉源县重点流域水环境综合治理项目' '雅安市 汉源县' '生态建设及环境保护项目' '生态修复、环境整治' '新建']]



### 数据处理主函数运行 (main)
> 组织各个步骤函数运行

In [53]:
from datetime import datetime

def mainDataPreprocess(pdf_path, out_dir, target_columns):
    # 提取表格数据
    raw_data = extract_table_from_pdf(pdf_path)
    
    # 转换为DataFrame
    df = convert_to_dataframe(raw_data, columns=target_columns)
    
    # 保存结果
    saved_path = save_results(df, pdf_path, out_dir)

    # 获取当前时间
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 输出结果信息
    print(f"解析完成({current_time})！共提取{len(df)}条项目数据，已保存至：")
    print(f"- CSV: {saved_path}")
    
    # 打印前5条数据验证结果
    print("\n解析结果预览（前5条）：")
    print(df.head())

# 运行主程序
if __name__ == "__main__":
    mainDataPreprocess(
        pdf_path="./assets/2024年四川省重点项目名单.pdf", 
        out_dir=CONST_OUTPUT_DIR,
        target_columns=CONST_TARGET_HEADERS
    )

    mainDataPreprocess(
        pdf_path="./assets/2025年四川省重点项目名单.pdf", 
        out_dir=CONST_OUTPUT_DIR,
        target_columns=CONST_TARGET_HEADERS
    )

解析完成！总共 76 页。
(727, 5)
Index(['项目名称', '建设地址', '一级项目领域', '二级项目领域', '建设性质'], dtype='object')
RangeIndex(start=0, stop=727, step=1)
项目名称      object
建设地址      object
一级项目领域    object
二级项目领域    object
建设性质      object
dtype: object
[['国家实验室' '四川 天府新区' '基础设施项目' '新型基础设施' '续建']
 ['极深地下极低辐射本底前沿物理实验设施' '凉山州 冕宁县' '基础设施项目' '新型基础设施' '续建']
 ['光场调控装置及地方配套项目' '四川 天府新区' '基础设施项目' '新型基础设施' '续建']
 ...
 ['西昌市两河口再生水处理厂工程' '凉山州 西昌市' '生态建设及环境保护项目' '生态修复、环境整治' '新建']
 ['越西河流域河道综合治理工程' '凉山州 越西县' '生态建设及环境保护项目' '生态修复、环境整治' '新建']
 ['汉源县重点流域水环境综合治理项目' '雅安市 汉源县' '生态建设及环境保护项目' '生态修复、环境整治' '新建']]
解析完成(2025-11-05 23:54:33)！共提取727条项目数据，已保存至：
- CSV: ./outputs/2024年四川省重点项目名单_解析结果.csv

解析结果预览（前5条）：
                 项目名称     建设地址  一级项目领域  二级项目领域 建设性质
0               国家实验室  四川 天府新区  基础设施项目  新型基础设施   续建
1  极深地下极低辐射本底前沿物理实验设施  凉山州 冕宁县  基础设施项目  新型基础设施   续建
2       光场调控装置及地方配套项目  四川 天府新区  基础设施项目  新型基础设施   续建
3      多态耦合轨道交通动模试验平台  四川 天府新区  基础设施项目  新型基础设施   续建
4           中国地震科学实验场   有关市（州）  基础设施项目  新型基础设施   续建
解析完成！总共 116 页。
(83